In [1]:
library(ggplot2)
library(dplyr)
library(readr)
library(progress)
if (!require("gridExtra")) {
  install.packages("gridExtra")
}
# if not install mvtnorm, install it
if (!require("mvtnorm")) {
  install.packages("mvtnorm")
}
library(mvtnorm)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: gridExtra

Warning message:
“package ‘gridExtra’ was built under R version 4.4.2”

Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Loading required package: mvtnorm



In [2]:
# create the output directory
base_path <- "./outputR_logOdds"
if (!dir.exists(base_path)) {
  dir.create(base_path, recursive = TRUE)
}

In [3]:

# fix the random seed, ensuring reproducibility
set.seed(1)

# set the parameters
sigma <- 2

In [4]:
# read and preprocess the image
img <- read_csv("https://raw.githubusercontent.com/aqlkzf/STAT6205-ProbabilisticMachineLearning-2025Spring/refs/heads/main/Assignment2/letterA.csv", 
                col_names = FALSE)
img <- as.matrix(img)
mean_val <- mean(img)
img2 <- ifelse(img >= mean_val, 1, -1)
# add noise to the image
y <- img2 + sigma * matrix(rnorm(length(img2)), nrow = nrow(img2), ncol = ncol(img2))

Rows: 128 Columns: 128
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (128): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
# load beta_s (logOdds) data
beta_s <- read_csv("https://github.com/aqlkzf/STAT6205-ProbabilisticMachineLearning-2025Spring/raw/refs/heads/main/Assignment2/beta_s.csv")
logOdds <- as.matrix(beta_s)

Rows: 128 Columns: 128
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (128): 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [6]:
# plot the original binary image and the noisy image
pdf(file.path(base_path, "denoising.pdf"), width = 10, height = 5)
par(mfrow = c(1, 2))
image(t(img2[nrow(img2):1,]), col = grey.colors(100), axes = FALSE, 
      main = "Original Binary Image")
image(t(y[nrow(y):1,]), col = grey.colors(100), axes = FALSE, 
      main = "Noisy Image")
dev.off()


pdf 
  2

 ## Sigmoid Function

 The sigmoid function is defined as:

 $$\sigma(u) = \frac{1}{1+e^{-u}}$$

 This function maps any real-valued number to a value between 0 and 1, which can be interpreted as a probability.
 In the context of the Ising model, it's used to compute the probability of a pixel taking the value +1.


In [7]:
# define sigmoid function
sigmoid <- function(x) {
  1 / (1 + exp(-x))
}

## Neighbor Summation

This function calculates the total contribution of a pixel’s neighbors at position (ix, iy). It computes the neighbor sum using the formula:

$$\eta_s = \sum_{t \in \text{Nbr}(s)} \beta_{st} \cdot z_t$$

Where:
- $\text{Nbr}(s)$ represents the 4-connected neighbors (up, down, left, right) of pixel $s$
- $\beta_{st}$ is the interaction strength between pixels $s$ and $t$ (set to $J$ in this implementation)
- $z_t$ is the current state of neighbor pixel $t$

The function returns $2 \cdot J \cdot \sum_{t \in \text{Nbr}(s)} z_t$, which is used in both Gibbs sampling and mean field updates.


In [8]:
# NeighborSum function: calculate the contribution of the pixel neighborhood (4-neighborhood) and multiply by the coefficient 2J
NeighborSum <- function(ix, iy, X, J) {
  wi <- 0
  if (iy > 1) {
    wi <- wi + X[iy - 1, ix]  # Top neighbor
  }
  if (iy < nrow(X)) {
    wi <- wi + X[iy + 1, ix]  # Bottom neighbor
  }
  if (ix > 1) {
    wi <- wi + X[iy, ix - 1]  # Left neighbor
  }
  if (ix < ncol(X)) {
    wi <- wi + X[iy, ix + 1]  # Right neighbor
  }
  return(2 * J * wi)
}



## Gibbs Sampling Algorithm

Gibbs sampling is a Markov Chain Monte Carlo (MCMC) method used to sample from the posterior distribution in the Ising model.

### Algorithm:
1. **Initialization**:  
   Set each pixel $z_s$ by thresholding the noisy image:
   $$z_s^{(0)}=\begin{cases}
   +1 & \text{if } \text{img}_s \geq \text{mean(img)},\\
   -1 & \text{otherwise}.
   \end{cases}$$

2. **Iterative Updates**:
   - Randomly select a pixel $s \in V$
   - Compute the local field: $\eta_s = \sum_{t \in \text{Nbr}(s)} \beta_{st} \cdot z_t + \beta_s$
   - Compute the probability: $\theta_s = \sigma(2\eta_s)$, where $\sigma(u)=\frac{1}{1+e^{-u}}$
   - Sample the state of $s$ as:
     $$z_s=\begin{cases}
     +1 & \text{with probability }\theta_s,\\
     -1 & \text{with probability }1-\theta_s.
     \end{cases}$$

3. **Output**:
   - Return the final denoised binary image (each pixel is either +1 or -1).

In this implementation:
- $\beta_{st} = J = 1$ for all neighboring pixels
- $\beta_s$ is provided by the logOdds values (precomputed)



In [9]:
# Gibbs sampling function
gibbs <- function(img, J, niter = 10, pre_computed_logOdds = NULL) {
  if (niter == 0) {
    return(img)
  }
    
  # initialize the binary image using the mean value of the original image
  mean_img <- mean(img)
  img2 <- ifelse(img >= mean_img, 1, -1)
  # use the pre-computed logOdds, otherwise calculate on the fly
  if (is.null(pre_computed_logOdds)) {
    # print error
    stop("Error: pre_computed_logOdds is NULL. Please provide precomputed logOdds values.")
 } else {
    logOdds_local <- pre_computed_logOdds
  }
  
  
  pb <- progress_bar$new(
    format = "Gibbs Sampling [:bar] :percent ETA: :eta",
    total = niter, 
    clear = FALSE, 
    width = 60
  )
  
  for (iter in 1:niter) {
    pb$tick()
    # randomly select a pixel position
    ix <- sample(1:ncol(img), 1)
    iy <- sample(1:nrow(img), 1)
    
    neighbor_sum <- NeighborSum(ix, iy, img2, J)  # Calculate NeighborSum
    
    # Update pixel value based on conditional probability
    if (runif(1) < sigmoid(neighbor_sum + logOdds_local[iy, ix])) {
      img2[iy, ix] <- 1
    } else {
      img2[iy, ix] <- -1
    }
    
}
  return(img2)
}


 ## Mean Field Variational Inference Algorithm

 Mean field variational inference approximates the posterior distribution by a factorized distribution where each pixel is independent.

 ### Algorithm:
 1. **Initialization**:  
    Set each pixel $\mu_s$ by thresholding the noisy image:
    $$\mu_s^{(0)}=\begin{cases}
    +1 & \text{if } \text{img}_s \geq \text{mean(img)},\\
    -1 & \text{otherwise}.
    \end{cases}$$

 2. **Iterative Updates**:
    - Randomly select a pixel $s \in V$
    - Update the mean for pixel $s$ via:
      $$\mu_s^{(k)} \leftarrow \tanh\left(\sum_{t \in \text{Nbr}(s)} \beta_{st} \cdot \mu_t^{(k-1)} + \beta_s\right)$$
    where the hyperbolic tangent is defined as $\tanh(x)=\frac{e^x-e^{-x}}{e^x+e^{-x}}$

 3. **Output**:
    - The final values of $\mu$ represent the expected values of the pixels in the denoised image.

 In this implementation:
 - $\beta_{st} = J = 1$ for all neighboring pixels
 - $\beta_s$ is provided by the logOdds values (precomputed)
 - A damping factor (rate) is applied to stabilize the updates

In [10]:
# Mean Field update function
meanfield <- function(img, J, niter = 10, pre_computed_logOdds = NULL) {
  mean_img <- mean(img)
  img2 <- ifelse(img >= mean_img, 1, -1) 
  if (is.null(pre_computed_logOdds)) {
    # print error
    stop("Error: pre_computed_logOdds is NULL. Please provide precomputed logOdds values.")
  } else {
    logOdds_local <- pre_computed_logOdds
  }
  
  # initialize the mean field state using the binary image
  mu <- matrix(img2, nrow = nrow(img2), ncol = ncol(img2))
  
  pb <- progress_bar$new(
    format = "Mean Field [:bar] :percent ETA: :eta",
    total = niter,
    clear = FALSE,
    width = 60
  )
  
  for (iter in 1:niter) {
    pb$tick()
    # randomly select a pixel to update
    ix <- sample(1:ncol(img), 1)
    iy <- sample(1:nrow(img), 1)
    
    neighbor_sum <- NeighborSum(ix, iy, mu, J)  # Calculate NeighborSum
    # Update mean field estimate with damped update
    mu[iy, ix] <- tanh(neighbor_sum +  logOdds_local[iy, ix])
  }
  
  return(mu)
}

In [11]:
# set the random seed for sampling
set.seed(10)

# define the iteration counts (in terms of pixel updates)
iters <- c(82000, 163000, 245000, 330000, 500000, 600000)
cat("Iteration counts:", iters, "\n")


Iteration counts: 82000 163000 245000 330000 5e+05 6e+05 


In [12]:
# plot the Mean Field denoising results
pdf(file.path(base_path, "meanFieldDenoising.pdf"), width = 4 * length(iters), height = 7)
par(mfrow = c(1, length(iters)), mar = c(2, 2, 6, 2))
for (i in seq_along(iters)) {
  result <- meanfield(y, J = 1, niter = iters[i], pre_computed_logOdds = logOdds)
  image(t(result[nrow(result):1,]), col = grey.colors(256), axes = FALSE, 
        main = paste("iter =", iters[i]))
}
mtext("Mean Field (pixel-level updates)", side = 3, outer = TRUE, line = -1.5, cex = 1.6)
dev.off()

pdf 
  2

In [13]:
# plot the Gibbs Sampling denoising results
pdf(file.path(base_path, "gibbsDenoising.pdf"), width = 4 * length(iters), height = 6)
par(mfrow = c(1, length(iters)), mar = c(2, 2,  6, 2))
for (i in seq_along(iters)) {
  result <- gibbs(y, J = 1, niter = iters[i], pre_computed_logOdds = logOdds)
  image(t(result[nrow(result):1,]), col = grey.colors(256), axes = FALSE, 
        main = paste("iter =", iters[i]))
}
mtext("Gibbs Sampling (pixel-level updates)", side = 3, outer = TRUE, line = -1.5, cex = 1.6)
dev.off()

pdf 
  2